In [1]:
import sys
print(sys.version)
print("executable: ")
print(sys.executable)
import tensorflow as tf
print(tf.__version__)
import tensorflow_datasets as tfds
print(tfds.__version__)
#print("TFDS version:", tfds.__version__)
from tensorflow.keras import layers, models
import numpy as np

3.8.10 (default, Nov 22 2023, 10:22:35) 
[GCC 9.4.0]
executable: 
/usr/bin/python


2025-01-31 23:10:22.929466: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-31 23:10:23.044242: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-31 23:10:24.246452: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-01-31 23:10:24.246509

2.11.0
4.9.2


In [4]:
mnist_train, mnist_test = tfds.load('mnist', split=['train', 'test'], as_supervised=True)
def preprocess_image(image, label):
    image = image
    #image = tf.cast(image, tf.float32) / 255.0  
    label = tf.one_hot(label, depth=10)  # One-hot encode the label
    return image, label
mnist_train = mnist_train.map(preprocess_image)
mnist_test = mnist_test.map(preprocess_image)
num_train_samples = len(list(mnist_train))
mnist_train = mnist_train.take(num_train_samples // 2)

In [5]:
#defining the network
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) *n_conv2

In [6]:
#initialising the weights with random values
# NOT USED IN THIS CODE WITH KERAS IMPLEMENTATION
weights = {
    "wc1" : tf.Variable(tf.random.normal([conv1_k, conv1_k, input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random.normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    "wh1" : tf.Variable(tf.random.normal([input_size_to_hidden, n_hidden])),
    "wo" : tf.Variable(tf.random.normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random.normal([n_conv1])),
    "bc2" : tf.Variable(tf.random.normal([n_conv2])),
    "bh1" : tf.Variable(tf.random.normal([n_hidden])),
    "bo" : tf.Variable(tf.random.normal([n_out])),
}

In [7]:
#functions for layers
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [8]:
#function for forward prop
#This function is edited now with keras functionality. weights are now handled internally but can still be accessed with get_weights method
def cnn(x):
    # Define the layers using Keras API
    #x = layers.Reshape((input_height, input_width, input_channels))(x)
    
    conv1 = layers.Conv2D(n_conv1, kernel_size=(conv1_k, conv1_k), strides=stride_conv1, padding="same", activation="relu")(x)
    conv1_pool = layers.MaxPooling2D(pool_size=(max_pool1_k, max_pool1_k))(conv1)

    conv2 = layers.Conv2D(n_conv2, kernel_size=(conv2_k, conv2_k), strides=stride_conv2, padding="same", activation="relu")(conv1_pool)
    conv2_pool = layers.MaxPooling2D(pool_size=(max_pool2_k, max_pool2_k))(conv2)

    # Flatten the output from convolutional layers
    flatten = layers.Flatten()(conv2_pool)

    # Fully connected layer
    hidden = layers.Dense(n_hidden, activation="relu")(flatten)

    # Dropout
    dropout = layers.Dropout(0.2)(hidden)

    # Output layer (logits)
    output = layers.Dense(n_out, activation = 'softmax')(dropout)
    
    return output

In [9]:
#x = tf.keras.Input(shape=(input_pixels,))
x = tf.keras.Input(shape=(input_height, input_width, input_channels))
y = tf.keras.Input(shape=(n_out,))
#pred is the model
pred = cnn(x)

In [10]:
model = models.Model(inputs=[x], outputs=pred)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) #changed from 0.01 to 0.001
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
train_data = mnist_train #it will be given in batches by the create_partly_adv_data method
test_data = mnist_test.batch(32)

In [11]:
# Added this cell for adversarial training

atn = tf.keras.models.load_model('ATN_base_model_2NDHALF.keras')

def create_partly_adv_data(atn, dataset, fraction=0.5):
    mnist_train_images = np.array([x.numpy() for x, y in dataset])
    mnist_train_labels = np.array([y.numpy() for x, y in dataset])
    num_samples = len(mnist_train_images)
    num_adversarial = int(fraction * num_samples)
    indices = np.random.choice(num_samples, num_adversarial, replace=False) #random should ensure every epoch new images are used to generate adversarial samples
    print(len(indices))
    adversarial_examples = atn.predict(mnist_train_images[indices])
    mnist_train_images[indices] = adversarial_examples
    # Optionally, recreate the tf.data.Dataset with the modified data
    updated_mnist_train = tf.data.Dataset.from_tensor_slices((mnist_train_images, mnist_train_labels)).batch(32) #return it in batches of 100
    return updated_mnist_train

In [12]:
num_epochs = 100
for i in range(num_epochs):
    print(f"Epoch {i + 1}/{num_epochs}")
    train_data_with_adv = create_partly_adv_data(atn, train_data, 0.5)
    model.fit(train_data_with_adv, epochs=1)

Epoch 1/100


KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(test_data) #dropout is automatically set to 1.0 when calling model.evaluate
print(f'Test accuracy: {test_acc}')
print("summary: ")
model.summary()

In [12]:
model.save('Baseline_AT.keras')